In [1]:
!pip3 install ../../random-forest-mc/

Processing /work/random-forest-mc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for random-forest-mc: filename=random_forest_mc-1.1.0-py3-none-any.whl size=14671 sha256=9f6b06361fae23145b4cc519d451454140e58fca65d7b5c6a473824f2c8ca1ef
  Stored in directory: /root/.cache/pip/wheels/eb/5f/19/4722d7f6bf0d13102d1af6ab98e4b53771da1ed44ce678108e
Successfully built random-forest-mc
  Attempting uninstall: random-forest-mc
    Found existing installation: random-forest-mc 1.1.0
    Uninstalling random-forest-mc-1.1.0:
      Successfully uninstalled random-forest-mc-1.1.0

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from random_forest_mc.model import RandomForestMC
from random_forest_mc.utils import LoadDicts
import pandas as pd
import numpy as np

from typing import Any
from typing import Dict
from typing import List
from typing import Optional
from typing import Tuple
from typing import Union

from numbers import Number
import logging as log

# Load dataset

In [3]:
dicts = LoadDicts("../../random-forest-mc/tests/")
dataset_dict = dicts.datasets_metadata
ds_name = "titanic"
params = dataset_dict[ds_name]
dataset = (
    pd.read_csv(params["csv_path"])[params["ds_cols"] + [params["target_col"]]]
    .dropna()
    .reset_index(drop=True)
)
dataset["Age"] = dataset["Age"].astype(np.uint8)
dataset["SibSp"] = dataset["SibSp"].astype(np.uint8)
dataset["Pclass"] = dataset["Pclass"].astype(str)
dataset["Fare"] = dataset["Fare"].astype(np.uint32)
ds_cols = params["ds_cols"]
target_col = params["target_col"]

# Load model

In [4]:
cls = RandomForestMC(target_col=target_col)
cls.fit(dataset)
row = dataset.reset_index(drop=True).loc[0]
predict_row = cls.predict(row)
predict_ds = cls.predict(dataset.sample(n=10))
predict_probs_ds = cls.predict_proba(dataset.sample(n=10))

Planting the forest: 100%|██████████| 16/16 [00:06<00:00,  2.53it/s]


In [5]:
predict_ds

['1', '0', '0', '1', '1', '1', '1', '0', '0', '0']

In [6]:
cls

RandomForestMC(len(Forest)=16,n_trees=16,model_version=1.1.0,module_version=1.1.0)

# Generate some missing data

In [7]:
df_tmp = dataset.sample(frac=0.2).reset_index(drop=True)
mask_random = np.random.choice([True, False], size=df_tmp[ds_cols].shape, p=[0.7, 0.3])
dataset_missing_values = df_tmp[ds_cols].mask(~mask_random)
dataset_missing_values[target_col] = df_tmp[target_col]
dataset_missing_values_ground_truth = df_tmp

In [8]:
dataset_missing_values

,Pclass,Sex,Age,SibSp,Fare,Embarked,Survived
0,1,female,16.0,0.0,NaN,C,1
1,3,male,11.0,NaN,NaN,S,0
2,NaN,male,28.0,0.0,NaN,S,0
3,NaN,male,42.0,1.0,NaN,S,0
4,2,female,28.0,1.0,NaN,S,1
...,...,...,...,...,...,...,...
137,NaN,male,26.0,1.0,NaN,NaN,0
138,NaN,male,24.0,2.0,73.0,S,0
139,NaN,male,NaN,NaN,7.0,NaN,0
140,NaN,female,NaN,1.0,7.0,NaN,0


In [9]:
dict_values = {col: dataset[col].unique().tolist() for col in ds_cols}

In [10]:
dict_values.keys()

dict_keys(['Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Embarked'])

In [11]:
df_pred_missings = cls.predictMissingValues(dataset_missing_values.loc[2], dict_values)

In [12]:
df_pred_missings

,Pclass,Sex,Age,SibSp,Fare,Embarked,Survived,0,1,row_id
0,NaN,male,28.0,0.0,NaN,S,0,NaN,NaN,0
1,3,male,28.0,0.0,NaN,S,0,0.8750,0.1250,0
2,1,male,28.0,0.0,NaN,S,0,0.6875,0.3125,0
3,2,male,28.0,0.0,NaN,S,0,0.6875,0.3125,0
4,NaN,male,28.0,0.0,7,S,0,0.6875,0.3125,0
...,...,...,...,...,...,...,...,...,...,...
87,NaN,male,28.0,0.0,133,S,0,0.6250,0.3750,0
88,NaN,male,28.0,0.0,25,S,0,0.6875,0.3125,0
89,NaN,male,28.0,0.0,37,S,0,0.5625,0.4375,0
90,NaN,male,28.0,0.0,50,S,0,0.6250,0.3750,0


In [14]:
df_pred_missings = cls.predictMissingValues(dataset_missing_values.sample(n=2), dict_values)
df_pred_missings

,Pclass,Sex,Age,SibSp,Fare,Embarked,Survived,0,1,row_id
0,2,NaN,34.0,NaN,10.0,S,1,NaN,NaN,0
1,2,male,34.0,NaN,10.0,S,1,0.6875,0.3125,0
2,2,female,34.0,NaN,10.0,S,1,0.1250,0.8750,0
3,2,NaN,34.0,1,10.0,S,1,0.3125,0.6875,0
4,2,NaN,34.0,0,10.0,S,1,0.2500,0.7500,0
5,2,NaN,34.0,3,10.0,S,1,0.3125,0.6875,0
6,2,NaN,34.0,4,10.0,S,1,0.3125,0.6875,0
7,2,NaN,34.0,2,10.0,S,1,0.3125,0.6875,0
8,2,NaN,34.0,5,10.0,S,1,0.3125,0.6875,0
9,1,female,40.0,0.0,153.0,S,1,NaN,NaN,1


In [15]:
dataset_missing_values

,Pclass,Sex,Age,SibSp,Fare,Embarked,Survived
0,1,female,16.0,0.0,NaN,C,1
1,3,male,11.0,NaN,NaN,S,0
2,NaN,male,28.0,0.0,NaN,S,0
3,NaN,male,42.0,1.0,NaN,S,0
4,2,female,28.0,1.0,NaN,S,1
...,...,...,...,...,...,...,...
137,NaN,male,26.0,1.0,NaN,NaN,0
138,NaN,male,24.0,2.0,73.0,S,0
139,NaN,male,NaN,NaN,7.0,NaN,0
140,NaN,female,NaN,1.0,7.0,NaN,0


In [16]:
df_pred_missings = cls.predictMissingValues(dataset_missing_values.sample(n=20), dict_values)
df_pred_missings

,Pclass,Sex,Age,SibSp,Fare,Embarked,Survived,0,1,row_id
0,3,male,NaN,0.0,NaN,Q,1,NaN,NaN,0
1,3,male,22,0.0,NaN,Q,1,0.6875,0.3125,0
2,3,male,38,0.0,NaN,Q,1,0.9375,0.0625,0
3,3,male,26,0.0,NaN,Q,1,0.6875,0.3125,0
4,3,male,35,0.0,NaN,Q,1,0.9375,0.0625,0
...,...,...,...,...,...,...,...,...,...,...
1588,NaN,female,50.0,NaN,NaN,C,1,0.3750,0.6250,18
1589,NaN,female,50.0,NaN,NaN,Q,1,0.3750,0.6250,18
1590,3,NaN,22.0,0.0,9.0,S,0,NaN,NaN,19
1591,3,male,22.0,0.0,9.0,S,0,0.8750,0.1250,19


In [17]:
df_pred_missings = cls.predictMissingValues(dataset.sample(n=20), dict_values)
df_pred_missings

MissingValuesNotFound: Dataset or row without missing values! Please, give a dataset or row with missing values using 'NaN'.

In [18]:
df_pred_missings = cls.predictMissingValues(dataset.loc[0], dict_values)
df_pred_missings

MissingValuesNotFound: Dataset or row without missing values! Please, give a dataset or row with missing values using 'NaN'.